In [142]:
#今回取得する情報; 物件名　住所　間取り　家賃　階数
#前提条件; 通勤時間が東京駅から20分になる物件

#基本
import pandas as pd
import numpy as up
import requests

#Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager

#Beautifulsoup(一応)
from bs4 import BeautifulSoup

#request(一応)
import urllib.request as req

#time
from time import sleep
from tqdm import tqdm

#Google_spread_sheet
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
import gspread


#固定情報
URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&ta=13&bs=040&ekInput=25620&tj=20&nk=-1&ct=9999999&cb=0.0&et=9999999&mt=9999999&mb=0&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&pc=30'

#time_start = time.time()        
#検索(ブラウザ非表示)
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options = options)
sleep(2)
        
browser.get(URL)
sleep(5)
page_num = int(browser.find_elements(By.TAG_NAME, 'ol')[1].text.split()[-1])

#データ格納用
bukken_uppers = []
bukken_names = []
bukken_addresses = []
bukken_status1s = []
bukken_status2s = []
bukken_status3s = []
bukken_years = []
bukken_totalfloors = []
bukken_floors = []
bukken_rents = []
bukken_managementfees = []
bukken_deposits = []
bukken_granuityfees = []
bukken_houselayouts = []
bukken_exclusiveareas = []

#今回は数が多いので一旦5ページ分
for l in tqdm(range(5)):
    l = l + 1
    _URL = URL + '&page=' + str(l)
    
    browser.get(_URL)
    sleep(5)
    bukken_list = browser.find_element(By.ID, 'js-bukkenList')
    
    for i in range(0, len(bukken_list.find_elements(By.CLASS_NAME, 'l-cassetteitem'))):
        bukken_lists = bukken_list.find_elements(By.CLASS_NAME, 'l-cassetteitem')[i]
        for k in range(0, len(bukken_list.find_elements(By.CLASS_NAME, 'cassetteitem'))):
            bukken = bukken_list.find_elements(By.CLASS_NAME, 'cassetteitem')[k]
                    
            #上部分
            bukken_upper = bukken.find_element(By.CLASS_NAME, 'cassetteitem-detail')
            bukken_name = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_content-title').text
            bukken_address = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_detail-col1').text
            bukken_status = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_detail-col2').text.split('\n')
            bukken_status1 = bukken_status[0]
            bukken_status2 = bukken_status[1] if len(bukken_status) == 3 else None
            bukken_status3 = bukken_status[2] if len(bukken_status) == 3 else None
            bukken_year, bukken_totalfloor = bukken_upper.find_element(By.CLASS_NAME, 'cassetteitem_detail-col3').text.split('\n')
            #print(bukken_upper)
            #print(bukken_name)
            #print(bukken_address)
            #print(bukken_status1)
            #print(bukken_status2)
            #print(bukken_status3)
            #print(bukken_year)
            #print(bukken_totalfloor)
        
            #下部分
            bukken_bottom = bukken.find_element(By.CLASS_NAME, 'cassetteitem-item')
            for j in range(0, len(bukken_bottom.find_elements(By.TAG_NAME, 'tbody'))):
                bukken_item = bukken_bottom.find_elements(By.TAG_NAME, 'tbody')[j]
                bukken_floor = bukken_item.find_elements(By.TAG_NAME, 'td')[2].text
                bukken_rent, bukken_managementfee = bukken_item.find_elements(By.TAG_NAME, 'td')[3].text.split('\n')
                bukken_deposit, bukken_gratuityfee = bukken_item.find_elements(By.TAG_NAME, 'td')[4].text.split('\n')
                bukken_houselayout, bukken_exclusivearea = bukken_item.find_elements(By.TAG_NAME, 'td')[5].text.split('\n')
                #print(bukken_floor)
                #print(bukken_rent)
                #print(bukken_managementfee)
                #print(bukken_deposit)
                #print(bukken_gratuityfee)
                #print(bukken_houselayout)
                #print(bukken_exclusivearea)
            
                bukken_names.append(bukken_name)
                bukken_addresses.append(bukken_address)
                bukken_status1s.append(bukken_status1)
                bukken_status2s.append(bukken_status2)
                bukken_status3s.append(bukken_status3)
                bukken_years.append(bukken_year)
                bukken_totalfloors.append(bukken_totalfloor)
                bukken_floors.append(bukken_floor)
                bukken_rents.append(bukken_rent)
                bukken_managementfees.append(bukken_managementfee)
                bukken_deposits.append(bukken_deposit)
                bukken_granuityfees.append(bukken_gratuityfee)
                bukken_houselayouts.append(bukken_houselayout)
                bukken_exclusiveareas.append(bukken_exclusivearea)
                        
df = pd.DataFrame({
    'Name': bukken_names,
    'Address': bukken_addresses,
    'Status1': bukken_status1s,
    'Status2': bukken_status2s,
    'Status3': bukken_status3s,
    'Year': bukken_years,
    'Total Floor': bukken_totalfloors,
    'Floor': bukken_floors,
    'Rent': bukken_rents,
    'Management Fee': bukken_managementfees,
    'Deposit': bukken_deposits,
    'Granuity Fee': bukken_granuityfees,
    'House Layout': bukken_houselayouts,
    'Exclusive Area': bukken_exclusiveareas
})

df['Check'] = df['Address'] + df['Year'] + df['Floor'] + df['House Layout'] + df['Exclusive Area']

100%|█████████████████████████████████████████████| 5/5 [02:22<00:00, 28.51s/it]


In [143]:
df

,Name,Address,Status1,Status2,Status3,Year,Total Floor,Floor,Rent,Management Fee,Deposit,Granuity Fee,House Layout,Exclusive Area,Check
0,東京メトロ東西線 葛西駅 8階建 築11年,東京都江戸川区南葛西４,東京メトロ東西線/葛西駅 歩23分,ＪＲ京葉線/葛西臨海公園駅 歩20分,ＪＲ京葉線/舞浜駅 歩39分,築11年,8階建,7階,15万円,9000円,15万円,15万円,2LDK,60.3m2,東京都江戸川区南葛西４築11年7階2LDK60.3m2
1,広尾ガーデンヒルズ,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩4分,都営大江戸線/六本木駅 歩20分,東京メトロ千代田線/乃木坂駅 歩20分,築41年,地下1地上9階建,1階,35万円,-,70万円,35万円,1LDK,65.99m2,東京都渋谷区広尾４築41年1階1LDK65.99m2
2,元赤坂タワーズ,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ丸ノ内線/四ツ谷駅 歩13分,東京メトロ千代田線/赤坂駅 歩14分,築45年,地下2地上13階建,2階,55万円,-,110万円,55万円,3LDK,118.26m2,東京都港区元赤坂１築45年2階3LDK118.26m2
3,元赤坂タワーズ,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ丸ノ内線/四ツ谷駅 歩13分,東京メトロ千代田線/赤坂駅 歩14分,築45年,地下2地上13階建,13階,130万円,-,260万円,130万円,3LDK,202.12m2,東京都港区元赤坂１築45年13階3LDK202.12m2
4,広尾ガーデンヒルズＪ棟,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩5分,都営大江戸線/六本木駅 歩18分,東京メトロ千代田線/乃木坂駅 歩19分,築39年,14階建,5階,30万円,-,60万円,30万円,1LDK,55.59m2,東京都渋谷区広尾４築39年5階1LDK55.59m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707,東京メトロ銀座線 赤坂見附駅 地下1地上7階建 築48年,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩6分,東京メトロ千代田線/赤坂駅 歩13分,東京メトロ有楽町線/永田町駅 歩7分,築48年,地下1地上7階建,5階,20.5万円,5000円,20.5万円,20.5万円,ワンルーム,46.78m2,東京都港区元赤坂１築48年5階ワンルーム46.78m2
2708,東京メトロ銀座線 赤坂見附駅 地下1地上7階建 築48年,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩6分,東京メトロ千代田線/赤坂駅 歩13分,東京メトロ有楽町線/永田町駅 歩7分,築48年,地下1地上7階建,4階,23万円,5000円,23万円,23万円,1LDK,47.53m2,東京都港区元赤坂１築48年4階1LDK47.53m2
2709,東京メトロ銀座線 赤坂見附駅 地下1地上7階建 築48年,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩6分,東京メトロ千代田線/赤坂駅 歩13分,東京メトロ有楽町線/永田町駅 歩7分,築48年,地下1地上7階建,4階,23万円,5500円,23万円,23万円,1LDK,47.53m2,東京都港区元赤坂１築48年4階1LDK47.53m2
2710,ヴェラハイツ日本橋蛎殻町,東京都中央区日本橋蛎殻町１,東京メトロ日比谷線/人形町駅 歩6分,東京メトロ半蔵門線/水天宮前駅 歩4分,東京メトロ東西線/茅場町駅 歩5分,築41年,8階建,6階,7.7万円,-,7.7万円,-,ワンルーム,22.53m2,東京都中央区日本橋蛎殻町１築41年6階ワンルーム22.53m2


In [144]:
#重複削除後の格納先dataframe
df_rent_min = pd.DataFrame()

#重複有無含めて1つだけ抽出_Check列
df_duplicated_first = df.drop_duplicates(keep = 'first', subset = 'Check')
df_duplicated_first_list = df_duplicated_first['Check'].to_list()

#各Check列のkeyから一番Rent(家賃)の安いもののみ抽出_for文
for m in range(0, len(df_duplicated_first_list)):
    check_key = df_duplicated_first_list[m]
    df_query = df.query('Check in @check_key')
    df_query_replace = df_query[['Rent']].replace('万円', '', regex=True).astype('float').rename(columns = {'Rent': 'Rent_int'}) * 10000
    df_query_int = pd.concat([df_query, df_query_replace], axis = 1)
    df_query_min = df_query_int.loc[[df_query_int['Rent_int'].idxmin()]]
    
    df_rent_min = pd.concat([df_rent_min, df_query_min])

df_rent_min = df_rent_min.reset_index(drop=True)

In [145]:
df_rent_min

,Name,Address,Status1,Status2,Status3,Year,Total Floor,Floor,Rent,Management Fee,Deposit,Granuity Fee,House Layout,Exclusive Area,Check,Rent_int
0,東京メトロ東西線 葛西駅 8階建 築11年,東京都江戸川区南葛西４,東京メトロ東西線/葛西駅 歩23分,ＪＲ京葉線/葛西臨海公園駅 歩20分,ＪＲ京葉線/舞浜駅 歩39分,築11年,8階建,7階,15万円,9000円,15万円,15万円,2LDK,60.3m2,東京都江戸川区南葛西４築11年7階2LDK60.3m2,150000.0
1,広尾ガーデンヒルズ,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩4分,都営大江戸線/六本木駅 歩20分,東京メトロ千代田線/乃木坂駅 歩20分,築41年,地下1地上9階建,1階,35万円,-,70万円,35万円,1LDK,65.99m2,東京都渋谷区広尾４築41年1階1LDK65.99m2,350000.0
2,元赤坂タワーズ,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ丸ノ内線/四ツ谷駅 歩13分,東京メトロ千代田線/赤坂駅 歩14分,築45年,地下2地上13階建,2階,55万円,-,110万円,55万円,3LDK,118.26m2,東京都港区元赤坂１築45年2階3LDK118.26m2,550000.0
3,元赤坂タワーズ,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ丸ノ内線/四ツ谷駅 歩13分,東京メトロ千代田線/赤坂駅 歩14分,築45年,地下2地上13階建,13階,130万円,-,260万円,130万円,3LDK,202.12m2,東京都港区元赤坂１築45年13階3LDK202.12m2,1300000.0
4,広尾ガーデンヒルズＪ棟,東京都渋谷区広尾４,東京メトロ日比谷線/広尾駅 歩5分,都営大江戸線/六本木駅 歩18分,東京メトロ千代田線/乃木坂駅 歩19分,築39年,14階建,5階,30万円,-,60万円,30万円,1LDK,55.59m2,東京都渋谷区広尾４築39年5階1LDK55.59m2,300000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,ロイヤル赤坂サルーン,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ半蔵門線/永田町駅 歩11分,東京メトロ千代田線/赤坂駅 歩12分,築48年,地下1地上7階建,2階,8.7万円,5000円,8.7万円,8.7万円,ワンルーム,22.13m2,東京都港区元赤坂１築48年2階ワンルーム22.13m2,87000.0
327,ロイヤル赤坂サルーン,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ半蔵門線/永田町駅 歩11分,東京メトロ千代田線/赤坂駅 歩12分,築48年,地下1地上7階建,5階,20.5万円,5000円,20.5万円,20.5万円,ワンルーム,46.78m2,東京都港区元赤坂１築48年5階ワンルーム46.78m2,205000.0
328,ロイヤル赤坂サルーン,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ半蔵門線/永田町駅 歩11分,東京メトロ千代田線/赤坂駅 歩12分,築48年,地下1地上7階建,7階,20.5万円,5000円,20.5万円,20.5万円,ワンルーム,46.78m2,東京都港区元赤坂１築48年7階ワンルーム46.78m2,205000.0
329,ロイヤル赤坂サルーン,東京都港区元赤坂１,東京メトロ銀座線/赤坂見附駅 歩8分,東京メトロ半蔵門線/永田町駅 歩11分,東京メトロ千代田線/赤坂駅 歩12分,築48年,地下1地上7階建,4階,23万円,5000円,23万円,23万円,1LDK,47.53m2,東京都港区元赤坂１築48年4階1LDK47.53m2,230000.0


In [146]:
#お決まりの2行
scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file("-------------json URL----------------", scopes = scope)

gc = gspread.authorize(credentials)

SPREADSHEET_KEY = '-----------------Sheet URL----------------------'
wb = gc.open_by_key(SPREADSHEET_KEY)
ws_1 = wb.worksheet('シート1')
ws_2 = wb.worksheet('シート2')

#ローデータ
df_to_sheet = [df.columns.values.tolist()] + df.values.tolist()
ws_1.update('A1', df_to_sheet)

#重複削除
df_rent_min_to_sheet = [df_rent_min.columns.values.tolist()] + df_rent_min.values.tolist()
ws_2.update('A1', df_rent_min_to_sheet)

/var/folders/xq/35rl0pc53_5dsrsn7dmxjx9c0000gn/T/ipykernel_19814/3101482069.py:14: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  ws_1.update('A1', df_to_sheet)
/var/folders/xq/35rl0pc53_5dsrsn7dmxjx9c0000gn/T/ipykernel_19814/3101482069.py:18: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  ws_2.update('A1', df_rent_min_to_sheet)


{'spreadsheetId': '1Jwe_8URWJ6e8lsXf0jW1-HAZqrTxXaSNBqk7MuZ0byM',
 'updatedRange': "'シート2'!A1:P332",
 'updatedRows': 332,
 'updatedColumns': 16,
 'updatedCells': 5308}